# Tweepy Streaming

In [2]:
import sys
sys.version

##1.1. Instantiate OAuthHandler

In [4]:
url_rest      = "https://api.twitter.com/1.1/search/tweets.json" 
url_streaming = "https://stream.twitter.com/1.1/statuses/sample.json" 

Set up tokens and secrets.

In [6]:
consumer_key        = u'rfbbqTwbEdU7FfhEM5qhWdQSn'
consumer_secret     = u'AXbEXOoIOKIRPPOGXizYCY8zrPpg73aznPwqlbiPBtqyNjX1ZZ'
access_token        = u'50464413-petgMmE6qEKzW33wN7AdMMZODxME9KJSSADyKx99r' 
access_token_secret = u'JBVgy7RcTnHswLyIaOgkLNMRiIsrICBt7fB4CxFRd5Wra' 

Instantiate OAuthHandler function

In [8]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

#Error handling
try:
  redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
  print ('Error! Failed to get request token.')

## 2.2. Stream Twitter data

Streaming or harvesting real time tweet data from Twitter is straighforward. When user makes request to stream, Twitter checks the authorization and opens streaming connection. When the connection is opened, Twitter server renders the tweets and user gets access to real time tweets.

![alt](https://cdn-images-1.medium.com/max/564/1*rPueOdOYwnsm8L3hST2ZbA.png)

`StreamListerner` function handles tweets that are received from the stream. This is a basic listener that just prints received tweets to stdout. The code below initiates list(`tweet_data`) and appends streaming tweets to the list.

In [13]:
from tweepy.streaming import StreamListener
from tweepy           import OAuthHandler
from tweepy           import Stream
from time             import strftime
from json             import loads

class StdOutListener(StreamListener):
  def __init__(self):
    self.tweet_data = []
    self.tweet_count = 0
        
  def on_data(self, data):
    self.tweet_count += 1
    print("Tweet", self.tweet_count, "downloaded")
    try:
      tweet = loads(data)
      self.tweet_data.append(tweet)
    except Exception as Argument:
      print ("Exception:\n", Argument)
    return True

  def on_error(self, status):
    print(status)

In [14]:
l = StdOutListener()

We use `Stream()` function streams the tweets in after authenticating our tokens. We can filter the tweets that contains certain words such as `trump`.

In [16]:
stream = Stream(auth, l)
stream.filter(track=['trump'])

In [17]:
tweets=l.tweet_data

Save `tweet_data` as pickle file so that we can access later.

In [19]:
import pickle
pickle.dump( l.tweet_data, open( "/dbfs/FileStore/tmp/stream_tweet_data.pkl", "wb" ) )

In [20]:
%sh ls -lh /dbfs/FileStore/tmp/*pkl

Loading tweet data as `tweets`.

In [22]:
import pickle
tweets = pickle.load(open( "/dbfs/FileStore/tmp/stream_tweet_data.pkl", "rb" ) )

`Tweets` file is a list consists of dictionaries.

In [24]:
len(tweets), type(tweets), type(tweets[0])

Each dictionary contains several dictionary.

In [26]:
tweets[0]['text']

Following elements are single value elements. `Text` is the tweet itself,

In [28]:
for key, val in tweets[0].items():
  if type(val) in [int, str, bool] or type(val)==None:
    print(key)

In [29]:
for key, val in tweets[0].items():
  if type(val) not in [int, str, bool] or type(val)!=None:
    print(key)

In [30]:
import pandas as pd

col_names=['text', 'id', 'created_at', 'favourites_count', 'retweet_count','lang']
tweets_data = pd.DataFrame.from_records(tweets, columns=col_names)

In [31]:
tweets_data['favourites_count']

In [32]:
tweets[0]['entities']

In [33]:
tweets[0]['retweeted_status']['text']

In [34]:
tweets[0]['user'].keys()

In [35]:
for key, val in tweets[0].items():
  if type(val) not in [int, str, bool]:
    if val!=None:
      print(key,":", val)

In [36]:
language=[]
index=[]
for i in range(len(tweets)):
  try: 
    if type(tweets[i]['lang'])==str:
      language.append(tweets[i]['lang'])
  except Exception as Argument:
    next
print(set(language))
print(len(set(language)))

In [37]:
index=[]
for i in range(len(tweets)):
  try: 
    if tweets[i]['lang']!='en':
      print(tweets[i]['retweeted_status']['text'])
      index.append(i)
  except Exception as Argument:
    next
print(index)

In [38]:
tweet=[]
for i in range(len(tweets)):
  try: 
    tweet.append(tweets[i]['retweeted_status']['text'])
  except Exception as Argument:
    next
print(len(tweet))
print(len(set(tweet)))

In [39]:
for i in range(len(tweets)):
  try:
    print(tweets[i]['retweeted_status']['created_at'])
  except Exception as Argument:
    print(Argument)

In [40]:
twitters=[]
for i in range(len(tweets)):
  try: 
    twitters.append(tweets[i]['retweeted_status']['user']['screen_name'])
  except Exception as Argument:
    next
print(len(twitters))
print(len(set(twitters)))

In [41]:
twitters_fol=[]
for i in range(len(tweets)):
  try: 
    twitters_fol.append(tweets[i]['retweeted_status']['user']['followers_count'])
  except Exception as Argument:
    next
print(len(twitters_fol))
print(len(set(twitters_fol)))

__Uyanga__: The following command doesn't work. It may be that not all tweets have the `'retweeted_status'` field (likely as not all tweets are retweets.) I reran the code above and so the `tweets` list has changed.